In [30]:
import pandas as pd
import sqlalchemy
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import TextBox
import scipy.stats as stats
from scipy.stats import ttest_ind
import statsmodels.api as sm
from statsmodels.formula.api import ols
from pathlib import Path
import plotly.graph_objects as go
import plotly.express as px
from ipywidgets import widgets, interactive
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output


In [3]:
#Read in the historic data and store in a DataFrame
path = "data/HistoricalPollution.csv"
historic_df = pd.read_csv(path)

#Take a look at the data
historic_df

,Primary Key,CBSA,City,State,Pollutant,Trend Statistic,Number of Sites,1990,1991,1992,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,1,10420,Akron,OH,O3,4th Max,2,0.090,0.101,0.087,...,0.059,0.060,0.065,0.060,0.066,0.068,0.062,0.063,0.067,0.070
1,2,10420,Akron,OH,PM2.5,Weighted Annual Mean,3,NaN,NaN,NaN,...,9.700,9.900,10.400,8.200,7.900,7.900,8.200,7.900,8.000,7.100
2,3,10420,Akron,OH,PM2.5,98th Percentile,3,NaN,NaN,NaN,...,24.000,22.000,23.000,17.000,18.000,18.000,21.000,21.000,20.000,17.000
3,4,10420,Akron,OH,SO2,99th Percentile,1,161.000,183.000,181.000,...,23.000,21.000,14.000,8.000,3.000,6.000,3.000,4.000,5.000,6.000
4,5,10500,Albany,GA,PM2.5,Weighted Annual Mean,1,NaN,NaN,NaN,...,10.000,10.300,9.000,8.700,9.400,8.400,9.300,9.100,10.100,8.300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
708,709,49700,Yuba City,CA,NO2,Annual Mean,1,17.000,17.000,17.000,...,10.000,8.000,7.000,7.000,7.000,7.000,6.000,6.000,5.000,7.000
709,710,49700,Yuba City,CA,NO2,98th Percentile,1,70.000,70.000,70.000,...,52.000,44.000,39.000,40.000,42.000,41.000,40.000,38.000,34.000,37.000
710,711,49700,Yuba City,CA,O3,4th Max,1,0.076,0.080,0.090,...,0.060,0.069,0.064,0.063,0.067,0.065,0.061,0.066,0.072,0.058
711,712,49700,Yuba City,CA,PM2.5,Weighted Annual Mean,1,NaN,NaN,NaN,...,8.200,9.400,9.600,8.100,9.300,10.300,8.400,16.400,14.500,10.700


In [4]:
#store all the years as a list to call easier
years = ['1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022']

In [5]:
#Create a dataframe containing state, city, and year total pollutiion

state_df = (historic_df.groupby(['State', 'City'], sort=True, as_index=False)[years].sum())
state_df


,State,City,1990,1991,1992,1993,1994,1995,1996,1997,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,AK,Anchorage,9.700,8.400,10.800,9.700,8.600,7.400,8.700,7.100,...,30.700,38.500,37.300,32.100,37.100,25.700,44.800,31.300,27.400,30.100
1,AK,Juneau,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,28.900,35.700,27.600,30.000,27.600,28.900,31.800,21.800,21.700,28.300
2,AL,Birmingham,142.493,157.173,148.484,181.183,139.977,121.498,99.994,117.984,...,87.065,84.364,113.268,73.670,70.562,69.265,66.471,61.961,68.861,63.064
3,AL,Daphne,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,25.400,25.900,26.600,21.200,26.400,24.100,22.500,24.900,22.300,22.300
4,AL,Decatur,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,24.600,30.300,25.800,20.400,23.700,23.400,21.600,23.100,33.000,29.500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,WY,Gillette,37.000,37.000,24.000,18.000,34.000,28.000,32.000,36.000,...,64.000,47.000,78.000,50.000,95.000,39.000,39.000,61.000,84.000,72.000
241,WY,Laramie,0.045,0.047,0.042,0.047,0.042,0.047,0.046,0.048,...,0.069,0.065,0.064,0.064,0.066,0.070,0.065,0.066,0.072,0.067
242,WY,Riverton,87.000,98.000,90.000,75.000,75.000,50.000,29.000,36.000,...,75.800,83.700,61.200,58.800,73.400,71.300,67.000,105.300,98.500,81.800
243,WY,Rock Springs,41.800,50.000,43.500,35.700,32.000,32.800,32.200,33.000,...,37.000,29.300,35.300,30.700,45.800,45.000,35.000,69.800,47.300,46.200


In [51]:
#Making a new dataframe from the above containing the averages for a future line graph
state_avg = state_df.groupby('State').mean().reset_index()
state_avg.head()

/var/folders/yd/nx3j_v1j3hj2pbltnq5y9bb40000gn/T/ipykernel_48772/3824548273.py:2: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



,State,1990,1991,1992,1993,1994,1995,1996,1997,1998,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,AK,4.850000,4.2000,5.400000,4.850000,4.300000,3.700000,4.350000,3.550000,4.200000,...,29.800000,37.100000,32.450000,31.050000,32.350000,27.300000,38.300000,26.550000,24.550000,29.200000
1,AL,25.095000,27.7115,24.592000,29.306375,23.715500,21.508750,18.120000,20.406375,21.689125,...,30.514000,32.127250,36.114625,27.252500,27.500375,27.114875,25.964250,27.372125,28.748875,27.363625
2,AR,0.050500,0.0480,0.053000,0.047000,0.048500,0.058000,0.054500,0.054000,0.053500,...,15.767000,16.162500,14.911500,14.259500,13.659000,15.214000,14.308000,13.157000,15.213500,14.511500
3,AZ,51.918400,42.4582,40.898200,41.258200,41.719400,47.800400,42.781200,49.617600,43.398600,...,70.117200,71.514800,47.254400,57.812600,62.915200,78.015400,46.892600,59.695600,71.015400,57.314000
4,CA,87.368143,86.8400,71.461107,71.010607,70.074679,73.083714,61.362036,60.860500,59.161286,...,79.676286,76.061071,71.457071,65.588286,84.593143,112.890643,65.724429,108.160893,83.341393,71.395857


In [7]:
state_avg.set_index('State', inplace=True)

In [8]:
#Transpose the data to build a line graph
state_avg_pose = state_avg.transpose()
state_avg_pose.head()

State,AK,AL,AR,AZ,CA,CO,CT,DE,FL,GA,...,SC,TN,TX,UT,VA,VT,WA,WI,WV,WY
1990,4.85,25.095000,0.0505,51.9184,87.368143,75.169833,29.110000,0.0,93.929750,15.021250,...,35.077333,2.84000,69.4105,42.361667,88.0790,37.719333,0.05000,15.190000,87.2324,33.1690
1991,4.20,27.711500,0.0480,42.4582,86.840000,70.037667,28.447000,0.0,73.391167,14.895000,...,35.070667,2.33475,69.2757,79.360667,91.0815,37.724333,0.04625,18.337429,102.2382,37.0094
1992,5.40,24.592000,0.0530,40.8982,71.461107,71.984000,26.425000,0.0,87.496500,14.894250,...,34.741000,3.58100,62.4444,69.691000,89.5780,38.918667,0.04800,15.903714,99.6256,31.5084
1993,4.85,29.306375,0.0470,41.2582,71.010607,78.750167,25.769333,0.0,70.589000,13.271125,...,23.077000,3.08925,66.4466,69.693333,106.0910,35.052667,0.04550,9.898000,86.2292,25.7494
1994,4.30,23.715500,0.0485,41.7194,70.074679,70.919167,32.096667,0.0,71.529250,11.518875,...,31.408667,5.08650,67.6758,34.691667,100.5835,31.752000,0.04675,10.186571,95.4338,28.2084


In [9]:
state_avg_pose.reset_index(inplace=True)

In [10]:
state_avg_pose.rename(columns={'index':'Year'})

State,Year,AK,AL,AR,AZ,CA,CO,CT,DE,FL,...,SC,TN,TX,UT,VA,VT,WA,WI,WV,WY
0,1990,4.85,25.095000,0.0505,51.9184,87.368143,75.169833,29.110000,0.0,93.929750,...,35.077333,2.84000,69.4105,42.361667,88.0790,37.719333,0.05000,15.190000,87.2324,33.1690
1,1991,4.20,27.711500,0.0480,42.4582,86.840000,70.037667,28.447000,0.0,73.391167,...,35.070667,2.33475,69.2757,79.360667,91.0815,37.724333,0.04625,18.337429,102.2382,37.0094
2,1992,5.40,24.592000,0.0530,40.8982,71.461107,71.984000,26.425000,0.0,87.496500,...,34.741000,3.58100,62.4444,69.691000,89.5780,38.918667,0.04800,15.903714,99.6256,31.5084
3,1993,4.85,29.306375,0.0470,41.2582,71.010607,78.750167,25.769333,0.0,70.589000,...,23.077000,3.08925,66.4466,69.693333,106.0910,35.052667,0.04550,9.898000,86.2292,25.7494
4,1994,4.30,23.715500,0.0485,41.7194,70.074679,70.919167,32.096667,0.0,71.529250,...,31.408667,5.08650,67.6758,34.691667,100.5835,31.752000,0.04675,10.186571,95.4338,28.2084
5,1995,3.70,21.508750,0.0580,47.8004,73.083714,64.785667,25.439000,0.0,63.718333,...,21.746667,3.59450,61.1855,34.361000,87.0840,26.584000,0.04525,10.621571,76.0372,22.1694
6,1996,4.35,18.120000,0.0545,42.7812,61.362036,63.401833,25.089000,0.0,61.117250,...,20.411667,3.09225,54.7548,47.028667,88.0785,26.581333,0.05050,9.474857,81.8332,18.6492
7,1997,3.55,20.406375,0.0540,49.6176,60.860500,56.550667,25.770333,0.0,61.671583,...,18.749000,3.34325,60.0464,38.359667,82.0900,32.818000,0.04450,13.758571,85.4320,21.0096
8,1998,4.20,21.689125,0.0535,43.3986,59.161286,45.738833,26.426667,0.0,67.006250,...,19.754667,2.85250,55.7203,25.026333,85.0910,26.516000,0.04725,8.761429,116.4368,24.3088
9,1999,3.90,23.934750,0.0605,47.8998,70.904786,57.601333,26.768000,0.0,65.749250,...,18.756667,4.59950,53.8600,30.359667,74.5920,18.487000,0.04225,9.336143,105.8398,21.3484


In [37]:
#Creating a dash app to display a line chart that shows overall pollution by year with a dropdown for each state

# Create a Dash app
appState = dash.Dash(__name__)


dropdown_options = [{'label': col, 'value': col} for col in state_avg_pose.columns[1:]]

# Define app layout
appState.layout = html.Div([
    dcc.Dropdown(
        id='dropdown',
        options=dropdown_options,
        value='AK'
    ),
    dcc.Graph(id='line-plot1')
])

# Define callback to update the line plot
@appState.callback(
    Output('line-plot1', 'figure'),
    [Input('dropdown', 'value')]
)
def update_line_plot(selected_column):
    fig = px.line(state_avg_pose, x='index', y=selected_column)

    return fig

# Run the app
if __name__ == '__main__':
    appState.run_server(debug=True, port=8051)

In [12]:
#Using the above table, perform a t-test for each state checking for significant difference at the 0.05 significance level between overall pollution in 1990 and 2022.

for state in state_df['State'].unique():
    data_1990 = state_df.loc[state_df['State'] == state, '1990']
    data_2022 = state_df.loc[state_df['State'] == state, '2022']

    t_statistic, p_value = ttest_ind(data_1990, data_2022)

    print(f'State: {state}')
    print(f't_statistic: {t_statistic}')
    print(f'p_value: {p_value}')
    if p_value < 0.05:
        print('There is a significant difference between pollution in 1990 and 2022')
    else:
        print('No significant difference between pollution in 1990 and 2022')

    print('-------------------------------')
    print('-------------------------------')



State: AK
t_statistic: -4.9363460764058535
p_value: 0.03867326921152213
There is a significant difference between pollution in 1990 and 2022
-------------------------------
-------------------------------
State: AL
t_statistic: -0.11357862409491835
p_value: 0.9111845088638956
No significant difference between pollution in 1990 and 2022
-------------------------------
-------------------------------
State: AR
t_statistic: -1.000655286369577
p_value: 0.42239759359520157
No significant difference between pollution in 1990 and 2022
-------------------------------
-------------------------------
State: AZ
t_statistic: -0.1255359675859275
p_value: 0.9031966845085668
No significant difference between pollution in 1990 and 2022
-------------------------------
-------------------------------
State: CA
t_statistic: 0.6730016772679176
p_value: 0.5038163162763069
No significant difference between pollution in 1990 and 2022
-------------------------------
-------------------------------
State: CO
t

/Users/jesserinskeys/anaconda3/envs/dev/lib/python3.10/site-packages/scipy/stats/_stats_py.py:1214: RuntimeWarning:

divide by zero encountered in divide

/Users/jesserinskeys/anaconda3/envs/dev/lib/python3.10/site-packages/scipy/stats/_stats_py.py:1214: RuntimeWarning:

invalid value encountered in scalar multiply

/var/folders/yd/nx3j_v1j3hj2pbltnq5y9bb40000gn/T/ipykernel_48772/3036444509.py:7: RuntimeWarning:

Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.



For the following we will look at each pollutant seperately.

In [13]:
#Create a dataframe containing only the pollutant O3
O3 = ['O3']
O3_df = historic_df[historic_df['Pollutant'].isin(O3) == True]
state_O3 = (O3_df.groupby(['State', 'City'], sort=True, as_index=False)[years].sum())
state_O3

,State,City,1990,1991,1992,1993,1994,1995,1996,1997,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,AL,Birmingham,0.093,0.073,0.084,0.083,0.077,0.098,0.094,0.084,...,0.065,0.064,0.068,0.070,0.062,0.065,0.071,0.061,0.061,0.064
1,AL,Fort Payne,0.023,0.009,0.013,0.022,0.022,0.026,0.028,0.030,...,0.062,0.062,0.065,0.064,0.058,0.064,0.062,0.057,0.058,0.061
2,AL,Huntsville,0.079,0.082,0.087,0.087,0.075,0.080,0.081,0.086,...,0.064,0.064,0.063,0.066,0.063,0.065,0.063,0.057,0.061,0.065
3,AL,Mobile,0.087,0.061,0.076,0.074,0.072,0.079,0.081,0.081,...,0.060,0.068,0.060,0.062,0.065,0.065,0.059,0.054,0.057,0.061
4,AL,Montgomery,0.078,0.067,0.076,0.085,0.078,0.087,0.076,0.070,...,0.061,0.060,0.061,0.058,0.055,0.060,0.059,0.048,0.054,0.058
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170,WI,Wausau,0.081,0.081,0.081,0.065,0.064,0.075,0.070,0.069,...,0.063,0.064,0.063,0.068,0.060,0.065,0.059,0.055,0.060,0.057
171,WI,Whitewater,0.077,0.095,0.086,0.079,0.081,0.086,0.083,0.081,...,0.067,0.073,0.067,0.072,0.065,0.070,0.062,0.074,0.069,0.070
172,WV,Charleston,0.079,0.090,0.055,0.063,0.075,0.091,0.078,0.075,...,0.067,0.067,0.067,0.069,0.067,0.066,0.065,0.057,0.065,0.061
173,WV,Parkersburg,0.083,0.101,0.073,0.083,0.094,0.095,0.088,0.085,...,0.064,0.067,0.071,0.067,0.058,0.061,0.061,0.058,0.057,0.062


In [71]:
O3_avg = state_O3.groupby('State').mean().reset_index()
O3_avg.head()

/var/folders/yd/nx3j_v1j3hj2pbltnq5y9bb40000gn/T/ipykernel_48772/1882764809.py:1: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



,State,1990,1991,1992,1993,1994,1995,1996,1997,1998,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,AL,0.072000,0.058400,0.067200,0.070200,0.064800,0.074000,0.072000,0.070200,0.082600,...,0.062400,0.063600,0.063400,0.064000,0.060600,0.063800,0.062800,0.055400,0.058200,0.061800
1,AR,0.050500,0.048000,0.053000,0.047000,0.048500,0.058000,0.054500,0.054000,0.053500,...,0.067000,0.062500,0.061500,0.059500,0.059000,0.064000,0.058000,0.057000,0.063500,0.061500
2,AZ,0.073000,0.072750,0.072750,0.072750,0.074250,0.075500,0.076500,0.072000,0.073250,...,0.071500,0.068500,0.068000,0.065750,0.069000,0.069250,0.065750,0.069500,0.069250,0.067500
3,CA,0.086095,0.086667,0.086238,0.085571,0.085286,0.087810,0.087476,0.075905,0.086476,...,0.068381,0.071905,0.071333,0.070095,0.071810,0.068476,0.065905,0.071667,0.069476,0.065905
4,CO,0.073000,0.075333,0.068000,0.067000,0.071667,0.071333,0.070333,0.068000,0.077667,...,0.075333,0.070333,0.072000,0.069333,0.070333,0.072333,0.066000,0.073667,0.077667,0.071667


In [14]:
#Using the above table, perform a t-test for each state checking for significant difference at the 0.05 significance level between O3 pollution in 1990 and 2022.
#If values return "nan" it means there was only one record for that state, which does not allow for a t-test.

for state in state_O3['State'].unique():
    data_1990 = state_O3.loc[state_O3['State'] == state, '1990']
    data_2022 = state_O3.loc[state_O3['State'] == state, '2022']

    t_statistic, p_value = ttest_ind(data_1990, data_2022)

    print(f'State: {state}')
    print(f't_statistic: {t_statistic}')
    print(f'p_value: {p_value}')
    if p_value < 0.05:
        print('There is a significant difference in O3 pollution between 1990 and 2022')
        if t_statistic > 0:
            print('There was a significant decrease in O3 pollution between 1990 and 2022')
        else:
            print('There was a significant increase in O3 pollution between 1990 and 2022')
            
    else:
        print('No significant difference in O3 pollution between 1990 and 2022')


    print('-------------------------------')
    print('-------------------------------')

State: AL
t_statistic: 0.8085567273852998
p_value: 0.4421494075410175
No significant difference in O3 pollution between 1990 and 2022
-------------------------------
-------------------------------
State: AR
t_statistic: -0.3724002529860428
p_value: 0.7453539960010644
No significant difference in O3 pollution between 1990 and 2022
-------------------------------
-------------------------------
State: AZ
t_statistic: 1.510966203435577
p_value: 0.1815461476982608
No significant difference in O3 pollution between 1990 and 2022
-------------------------------
-------------------------------
State: CA
t_statistic: 3.568194407742485
p_value: 0.0009515163995818402
There is a significant difference in O3 pollution between 1990 and 2022
There was a significant decrease in O3 pollution between 1990 and 2022
-------------------------------
-------------------------------
State: CO
t_statistic: 0.3429971702850194
p_value: 0.7488684500235252
No significant difference in O3 pollution between 1990 an

/var/folders/yd/nx3j_v1j3hj2pbltnq5y9bb40000gn/T/ipykernel_48772/2516471284.py:8: RuntimeWarning:

Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.

/Users/jesserinskeys/anaconda3/envs/dev/lib/python3.10/site-packages/scipy/stats/_stats_py.py:1214: RuntimeWarning:

divide by zero encountered in divide

/Users/jesserinskeys/anaconda3/envs/dev/lib/python3.10/site-packages/scipy/stats/_stats_py.py:1214: RuntimeWarning:

invalid value encountered in scalar multiply



In [15]:
#Create a dataframe containing only PM2.5 and continue for each pollutant
pm_2 = ['PM2.5']
pm25_df = historic_df[historic_df['Pollutant'].isin(pm_2) == True]
state_pm25 = (pm25_df.groupby(['State', 'City'], sort=True, as_index=False)[years].sum())
state_pm25

,State,City,1990,1991,1992,1993,1994,1995,1996,1997,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,AK,Anchorage,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,27.6,36.0,34.5,29.1,33.6,23.0,42.4,28.3,25.2,27.7
1,AK,Juneau,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,28.9,35.7,27.6,30.0,27.6,28.9,31.8,21.8,21.7,28.3
2,AL,Birmingham,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,31.3,32.9,30.3,26.2,29.5,28.0,27.2,27.8,32.8,28.2
3,AL,Daphne,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.4,25.9,26.6,21.2,26.4,24.1,22.5,24.9,22.3,22.3
4,AL,Decatur,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,24.6,30.3,25.8,20.4,23.7,23.4,21.6,23.1,33.0,29.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164,WV,Fairmont,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,27.3,27.5,28.5,25.6,22.7,22.4,23.6,23.4,22.3,22.1
165,WV,Morgantown,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,27.9,25.6,28.2,23.4,25.3,25.9,22.2,22.2,28.7,21.0
166,WV,Parkersburg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,30.4,27.6,34.3,21.9,26.1,24.7,23.7,22.0,26.9,22.4
167,WY,Riverton,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,36.8,32.7,26.2,28.8,34.4,29.3,33.0,44.3,37.5,20.8


In [16]:
SO2 = ['SO2']
SO2_df = historic_df[historic_df['Pollutant'].isin(SO2) == True]
state_SO2 = (SO2_df.groupby(['State', 'City'], sort=True, as_index=False)[years].sum())
state_SO2.head()

,State,City,1990,1991,1992,1993,1994,1995,1996,1997,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,AL,Birmingham,75.0,81.0,89.0,113.0,85.0,65.0,54.0,60.0,...,22.0,17.0,17.0,11.0,12.0,11.0,10.0,5.0,7.0,6.0
1,CA,San Francisco,30.0,25.0,20.0,25.0,25.0,30.0,21.0,25.0,...,9.0,12.0,8.0,9.0,9.0,10.0,10.0,8.0,9.0,7.0
2,CA,Santa Maria,13.0,13.0,15.0,13.0,6.0,3.0,7.0,9.0,...,3.0,4.0,2.0,2.0,1.0,2.0,3.0,3.0,2.0,2.0
3,CO,Denver,84.0,85.0,117.0,105.0,92.0,98.0,86.0,86.0,...,34.0,16.0,15.0,14.0,13.0,7.0,6.0,6.0,6.0,5.0
4,FL,Jacksonville,239.0,108.0,113.0,109.0,120.0,140.0,74.0,82.0,...,33.0,37.0,27.0,25.0,20.0,22.0,24.0,28.0,20.0,20.0


In [17]:
#Using the above table, perform a t-test for each state checking for significant difference at the 0.05 significance level between SO2 pollution in 1990 and 2022. 
#If values return "nan" it means there was only one record for that state, which does not allow for a t-test.

for state in state_SO2['State'].unique():
    data_1990 = state_SO2.loc[state_SO2['State'] == state, '1990']
    data_2022 = state_SO2.loc[state_SO2['State'] == state, '2022']

    t_statistic, p_value = ttest_ind(data_1990, data_2022)

    print(f'State: {state}')
    print(f't_statistic: {t_statistic}')
    print(f'p_value: {p_value}')
    if p_value < 0.05:
        print('There is a significant difference in SO2 pollution between 1990 and 2022')
        if t_statistic > 0:
            print('There was a significant decrease in SO2 pollution between 1990 and 2022')
        else:
            print('There was a significant increase in SO2 pollution between 1990 and 2022')
            
    else:
        print('No significant difference in SO2 pollution between 1990 and 2022')


    print('-------------------------------')
    print('-------------------------------')

State: AL
t_statistic: nan
p_value: nan
No significant difference in O3 pollution between 1990 and 2022
-------------------------------
-------------------------------
State: CA
t_statistic: 1.9187310031425413
p_value: 0.19502666952235664
No significant difference in O3 pollution between 1990 and 2022
-------------------------------
-------------------------------
State: CO
t_statistic: nan
p_value: nan
No significant difference in O3 pollution between 1990 and 2022
-------------------------------
-------------------------------
State: FL
t_statistic: 2.395663925289263
p_value: 0.03759311611049655
There is a significant difference in O3 pollution between 1990 and 2022
There was a significant decrease in O3 pollution between 1990 and 2022
-------------------------------
-------------------------------
State: GA
t_statistic: nan
p_value: nan
No significant difference in O3 pollution between 1990 and 2022
-------------------------------
-------------------------------
State: HI
t_statisti

/var/folders/yd/nx3j_v1j3hj2pbltnq5y9bb40000gn/T/ipykernel_48772/3386300237.py:8: RuntimeWarning:

Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.

/Users/jesserinskeys/anaconda3/envs/dev/lib/python3.10/site-packages/scipy/stats/_stats_py.py:1214: RuntimeWarning:

divide by zero encountered in divide

/Users/jesserinskeys/anaconda3/envs/dev/lib/python3.10/site-packages/scipy/stats/_stats_py.py:1214: RuntimeWarning:

invalid value encountered in scalar multiply



In [18]:
NO2 = ['NO2']
NO2_df = historic_df[historic_df['Pollutant'].isin(NO2) == True]
NO2_df

,Primary Key,CBSA,City,State,Pollutant,Trend Statistic,Number of Sites,1990,1991,1992,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
10,11,10740,Albuquerque,NM,NO2,Annual Mean,1,18.0,2.0,21.0,...,12.0,12.0,11.0,10.0,10.0,10.0,9.0,8.0,8.0,9.0
11,12,10740,Albuquerque,NM,NO2,98th Percentile,1,72.0,77.0,83.0,...,45.0,42.0,43.0,44.0,45.0,45.0,44.0,41.0,44.0,44.0
35,36,12060,Atlanta,GA,NO2,Annual Mean,1,15.0,14.0,14.0,...,9.0,11.0,10.0,11.0,9.0,8.0,9.0,8.0,8.0,8.0
44,45,12540,Bakersfield,CA,NO2,Annual Mean,2,16.0,17.0,17.0,...,10.0,9.0,8.0,8.0,7.0,8.0,7.0,7.0,7.0,7.0
45,46,12540,Bakersfield,CA,NO2,98th Percentile,2,65.0,65.0,60.0,...,42.0,41.0,35.0,32.0,33.0,33.0,34.0,30.0,30.0,29.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,696,49180,Winston,NC,NO2,98th Percentile,1,61.0,56.0,57.0,...,37.0,41.0,39.0,36.0,38.0,35.0,34.0,30.0,32.0,35.0
702,703,49620,York,PA,NO2,Annual Mean,1,22.0,21.0,20.0,...,10.0,11.0,10.0,10.0,8.0,6.0,7.0,7.0,8.0,9.0
703,704,49620,York,PA,NO2,98th Percentile,1,81.0,69.0,65.0,...,39.0,45.0,52.0,47.0,42.0,41.0,40.0,39.0,40.0,39.0
708,709,49700,Yuba City,CA,NO2,Annual Mean,1,17.0,17.0,17.0,...,10.0,8.0,7.0,7.0,7.0,7.0,6.0,6.0,5.0,7.0


In [19]:
CO = ['CO']
CO_df = historic_df[historic_df['Pollutant'].isin(CO) == True]
state_CO = (CO_df.groupby(['State', 'City'], sort=True, as_index=False)[years].sum())
state_CO

,State,City,1990,1991,1992,1993,1994,1995,1996,1997,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,AK,Anchorage,9.7,8.4,10.8,9.7,8.6,7.4,8.7,7.1,...,3.1,2.5,2.8,3.0,3.5,2.7,2.4,3.0,2.2,2.4
1,AL,Birmingham,6.4,6.1,7.4,7.1,6.9,6.4,4.9,5.9,...,1.7,2.4,1.9,1.4,1.0,1.2,1.2,1.1,1.0,0.8
2,AZ,Phoenix,6.6,6.7,7.6,6.4,7.4,7.4,6.5,6.2,...,2.0,2.1,1.9,1.9,2.2,2.0,1.7,1.9,1.9,1.7
3,AZ,Tucson,6.5,5.7,5.8,6.0,5.5,5.9,5.1,4.4,...,1.1,1.0,1.0,0.9,1.0,0.8,0.6,0.8,0.7,0.7
4,CA,Fresno,4.8,4.8,3.6,3.4,3.7,3.6,3.5,2.9,...,1.4,1.1,1.0,1.3,1.2,1.3,1.0,2.2,1.1,1.0
5,CA,Los Angeles,8.3,8.0,6.9,5.8,7.2,7.2,5.8,5.4,...,1.7,1.6,1.6,1.3,1.5,1.4,1.3,1.5,1.4,1.2
6,CA,Riverside,4.4,5.9,3.8,3.7,3.7,3.7,3.1,3.3,...,1.2,1.6,1.1,1.3,1.0,1.0,0.7,1.1,1.0,0.9
7,CA,Sacramento,9.7,7.8,6.9,7.4,7.4,5.4,5.0,5.7,...,2.1,1.7,1.9,1.7,1.6,3.3,1.2,2.0,1.1,1.1
8,CA,San Francisco,5.4,5.6,4.6,4.2,4.0,3.1,3.3,3.1,...,1.2,1.2,1.1,1.0,1.2,1.6,0.9,1.6,0.9,0.9
9,CA,Santa Maria,1.6,1.3,1.4,1.6,1.7,1.2,1.2,1.1,...,0.6,0.6,0.5,3.2,1.3,0.7,0.5,0.7,0.6,0.6


In [20]:
#Using the above table, perform a t-test for each state checking for significant difference at the 0.05 significance level between CO pollution in 1990 and 2022. 
#If values return "nan" it means there was only one record for that state, which does not allow for a t-test.

for state in state_CO['State'].unique():
    data_1990 = state_CO.loc[state_CO['State'] == state, '1990']
    data_2022 = state_CO.loc[state_CO['State'] == state, '2022']

    t_statistic, p_value = ttest_ind(data_1990, data_2022)

    print(f'State: {state}')
    print(f't_statistic: {t_statistic}')
    print(f'p_value: {p_value}')
    if p_value < 0.05:
        print('There is a significant difference in CO pollution between 1990 and 2022')
        if t_statistic > 0:
            print('There was a significant decrease in CO pollution between 1990 and 2022')
        else:
            print('There was a significant increase in CO pollution between 1990 and 2022')
            
    else:
        print('No significant difference in O3 pollution between 1990 and 2022')


    print('-------------------------------')
    print('-------------------------------')

State: AK
t_statistic: nan
p_value: nan
No significant difference in O3 pollution between 1990 and 2022
-------------------------------
-------------------------------
State: AL
t_statistic: nan
p_value: nan
No significant difference in O3 pollution between 1990 and 2022
-------------------------------
-------------------------------
State: AZ
t_statistic: 10.646897935246884
p_value: 0.008706686994593281
There is a significant difference in O3 pollution between 1990 and 2022
There was a significant decrease in O3 pollution between 1990 and 2022
-------------------------------
-------------------------------
State: CA
t_statistic: 4.705281174633279
p_value: 0.0005096693829078108
There is a significant difference in O3 pollution between 1990 and 2022
There was a significant decrease in O3 pollution between 1990 and 2022
-------------------------------
-------------------------------
State: CO
t_statistic: 5.949602467117328
p_value: 0.02710695616039529
There is a significant difference in

/var/folders/yd/nx3j_v1j3hj2pbltnq5y9bb40000gn/T/ipykernel_48772/668419569.py:8: RuntimeWarning:

Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.

/Users/jesserinskeys/anaconda3/envs/dev/lib/python3.10/site-packages/scipy/stats/_stats_py.py:1214: RuntimeWarning:

divide by zero encountered in divide

/Users/jesserinskeys/anaconda3/envs/dev/lib/python3.10/site-packages/scipy/stats/_stats_py.py:1214: RuntimeWarning:

invalid value encountered in scalar multiply



In [21]:
pm10 = ['PM10']
pm10_df = historic_df[historic_df['Pollutant'].isin(pm10) == True]
pm10_df.head()

,Primary Key,CBSA,City,State,Pollutant,Trend Statistic,Number of Sites,1990,1991,1992,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
13,14,10740,Albuquerque,NM,PM10,2nd Max,1,43.0,45.0,37.0,...,92.0,106.0,44.0,63.0,46.0,60.0,49.0,67.0,85.0,100.0
73,74,13820,Birmingham,AL,PM10,2nd Max,1,61.0,70.0,52.0,...,32.0,32.0,64.0,35.0,28.0,29.0,28.0,28.0,28.0,28.0
84,85,14500,Boulder,CO,PM10,2nd Max,1,84.0,69.0,69.0,...,44.0,55.0,42.0,41.0,38.0,57.0,50.0,65.0,51.0,39.0
113,114,16700,Charleston,SC,PM10,2nd Max,1,93.0,93.0,93.0,...,34.0,30.0,39.0,66.0,29.0,31.0,54.0,45.0,40.0,53.0
122,123,17460,Cleveland,OH,PM10,2nd Max,4,93.3,83.5,73.0,...,37.3,33.5,35.8,44.5,35.3,38.3,37.3,30.9,33.5,38.5


In [22]:

#create a dataframe summing the total pollutants in each city by year
city_total_df = (historic_df.groupby('City', sort=True, as_index=False)[years].sum())


In [23]:
#Convert list of cities to be used in a dropdown menu for future plots
city_list = city_total_df['City'].tolist()

['Akron',
 'Albany',
 'Albuquerque',
 'Altoona',
 'Americus',
 'Anchorage',
 'Ann Arbor',
 'Appleton',
 'Arkadelphia',
 'Asheville',
 'Ashtabula',
 'Athens',
 'Atlanta',
 'Atlantic City',
 'Augusta',
 'Austin',
 'Bakersfield',
 'Baltimore',
 'Barnstable Town',
 'Baton Rouge',
 'Bay City',
 'Beaumont',
 'Bellingham',
 'Bennington',
 'Billings',
 'Birmingham',
 'Bishop',
 'Bismarck',
 'Boulder',
 'Bowling Green',
 'Bridgeport',
 'Brunswick',
 'Buffalo',
 'Burlington',
 'Butte',
 'Canton',
 'Cape Coral',
 'Charleston',
 'Chico',
 'Clarksburg',
 'Clearlake',
 'Cleveland',
 'Clinton',
 'Columbia',
 'Columbus',
 'Concord',
 'Corning',
 'Corpus Christi',
 'Cullowhee',
 'Dallas',
 'Daphne',
 'Dayton',
 'Decatur',
 'Deltona',
 'Denver',
 'Des Moines',
 'Detroit',
 'Dickinson',
 'Dover',
 'Durango',
 'Durham',
 'Effingham',
 'El Centro',
 'El Paso',
 'Elizabethtown',
 'Erie',
 'Eugene',
 'Fairmont',
 'Fayetteville',
 'Flagstaff',
 'Flint',
 'Fort Collins',
 'Fort Payne',
 'Fort Wayne',
 'Fresno'

In [24]:
#Set index to prepare for dataframe inversion
city_total_df.set_index('City', inplace=True)

In [25]:
city_total_df.head()

,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
City,,,,,,,,,,,,,,,,,,,,,
Akron,161.090,183.101,181.087,209.093,132.086,123.092,133.091,155.087,143.097,158.097,...,56.759,52.960,47.465,33.260,28.966,31.968,32.262,32.963,33.067,30.170
Albany,0.086,0.086,0.086,0.082,0.080,0.079,0.077,0.080,0.079,0.088,...,63.163,59.561,57.862,53.068,56.361,56.966,54.355,61.954,65.661,49.059
Albuquerque,138.172,129.567,145.464,175.064,146.574,134.870,135.167,145.566,117.072,113.572,...,171.667,185.662,127.066,141.665,122.669,136.174,124.067,141.271,160.571,174.371
Altoona,119.081,100.092,94.079,76.086,103.092,70.091,75.083,73.096,59.094,71.091,...,29.065,30.060,31.069,13.062,7.063,8.064,8.064,4.063,4.061,6.062
Americus,0.082,0.082,0.081,0.080,0.076,0.079,0.082,0.085,0.081,0.084,...,0.060,0.059,0.057,0.065,0.058,0.062,0.062,0.054,0.059,0.059


In [26]:
#Transposing the dataframe
cities_df = city_total_df.transpose()
cities_df.head()


City,Akron,Albany,Albuquerque,Altoona,Americus,Anchorage,Ann Arbor,Appleton,Arkadelphia,Asheville,...,Washington Court House,Watertown,Wausau,Whitewater,Wichita,Wilmington,Winston,Yakima,York,Yuba City
1990,161.090,0.086,138.172,119.081,0.082,9.7,0.025,0.0,0.021,0.073,...,0.025,0.077,0.081,0.077,0.077,0.082,149.084,0.0,212.097,87.076
1991,183.101,0.086,129.567,100.092,0.082,8.4,0.030,0.0,0.018,0.063,...,0.027,0.105,0.081,0.095,0.072,0.083,169.085,0.0,193.100,87.080
1992,181.087,0.086,145.464,94.079,0.081,10.8,0.020,0.0,0.030,0.064,...,0.014,0.083,0.081,0.086,0.070,0.083,133.082,0.0,200.083,87.090
1993,209.093,0.082,175.064,76.086,0.080,9.7,0.021,0.0,0.018,0.066,...,0.031,0.080,0.065,0.079,0.059,0.083,164.092,0.0,190.090,87.078
1994,132.086,0.080,146.574,103.092,0.076,8.6,0.032,0.0,0.021,0.069,...,0.029,0.085,0.064,0.081,0.067,0.082,150.088,0.0,195.082,81.089


In [27]:
#reset the index
cities_df.reset_index(inplace=True)

In [28]:
cities_df.head()

City,index,Akron,Albany,Albuquerque,Altoona,Americus,Anchorage,Ann Arbor,Appleton,Arkadelphia,...,Washington Court House,Watertown,Wausau,Whitewater,Wichita,Wilmington,Winston,Yakima,York,Yuba City
0,1990,161.090,0.086,138.172,119.081,0.082,9.7,0.025,0.0,0.021,...,0.025,0.077,0.081,0.077,0.077,0.082,149.084,0.0,212.097,87.076
1,1991,183.101,0.086,129.567,100.092,0.082,8.4,0.030,0.0,0.018,...,0.027,0.105,0.081,0.095,0.072,0.083,169.085,0.0,193.100,87.080
2,1992,181.087,0.086,145.464,94.079,0.081,10.8,0.020,0.0,0.030,...,0.014,0.083,0.081,0.086,0.070,0.083,133.082,0.0,200.083,87.090
3,1993,209.093,0.082,175.064,76.086,0.080,9.7,0.021,0.0,0.018,...,0.031,0.080,0.065,0.079,0.059,0.083,164.092,0.0,190.090,87.078
4,1994,132.086,0.080,146.574,103.092,0.076,8.6,0.032,0.0,0.021,...,0.029,0.085,0.064,0.081,0.067,0.082,150.088,0.0,195.082,81.089


In [38]:
##Creating a dash app to display a line chart that shows overall pollution by year with a dropdown for each state

# Create a Dash app
app = dash.Dash(__name__)


dropdown_options = [{'label': col, 'value': col} for col in cities_df.columns[1:]]

# Define app layout
app.layout = html.Div([
    dcc.Dropdown(
        id='dropdown-column',
        options=dropdown_options,
        value='Akron'
    ),
    dcc.Graph(id='line-plot')
])

# Define callback to update the line plot
@app.callback(
    Output('line-plot', 'figure'),
    [Input('dropdown-column', 'value')]
)
def update_line_plot(selected_column):
    fig = px.line(cities_df, x='index', y=selected_column)

    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, port = 8050)

The following will include some statistics found from these above data starting with the state level.

In [49]:
state_avg.reset_index(inplace=True)


In [79]:
#What are the best and worst states for pollution?
lowest_pollutant = state_avg['2022'].min()
best_state = state_avg.loc[state_avg['2022'].idxmin(), 'State']
print(f'{best_state} is the state with the lowest overall pollution concentration with a pollutant density value of {lowest_pollutant}.')

most_pollutant = state_avg['2022'].max()
worst_state = state_avg.loc[state_avg['2022'].idxmax(), 'State']
print(f'{worst_state} is the state with the highest overall pollution concentration with a pollutant density value of {most_pollutant}.')

ME is the state with the lowest overall pollution concentration with a pollutant density value of 0.057666666666666665.
OR is the state with the highest overall pollution concentration with a pollutant density value of 145.52075.


In [95]:
#Which states improved and deproved over time the most in pollution concentration?

state_avg['Change_Over_Time'] = state_avg['2022'] - state_avg['1990']

best_change = state_avg['Change_Over_Time'].min()
best_change_state = state_avg.loc[state_avg['Change_Over_Time'].idxmin(), 'State']
print(f'{best_change_state} is the state with the best overall pollution change over time with a pollutant density change of {best_change}.')


worst_change = state_avg['Change_Over_Time'].max()
worst_change_state = state_avg.loc[state_avg['Change_Over_Time'].idxmax(), 'State']
print(f'{worst_change_state} is the state with the worst overall pollution change over time with a pollutant density change of {worst_change}.')



MT is the state with the best overall pollution change over time with a pollutant density change of -87.067.
OR is the state with the worst overall pollution change over time with a pollutant density change of 108.38825.


The following will take a look at the same statistics at the city level.

In [64]:
city_total_df.reset_index(inplace=True)

In [78]:
#What are the best and worst cities for pollution?
low_pollutant = city_total_df['2022'].min()
best_city = city_total_df.loc[city_total_df['2022'].idxmin(), 'City']
print(f'{best_city} is the city with the lowest overall pollution concentration with a value of {low_pollutant}.')

high_pollutant = city_total_df['2022'].max()
worst_city = city_total_df.loc[city_total_df['2022'].idxmax(), 'City']
print(f'{worst_city} is the city with the highest pollution concentration with a value of {high_pollutant}.')

Bellingham is the city with the lowest overall pollution concentration with a value of 0.048.
Eugene is the city with the highest pollution concentration with a value of 506.758.


In [98]:
#Which cities improved and deproved over time the most in pollution concentration?

city_total_df['Change_Over_Time'] = city_total_df['2022'] - city_total_df['1990']

best_change_c = city_total_df['Change_Over_Time'].min()
best_change_city = city_total_df.loc[city_total_df['Change_Over_Time'].idxmin(), 'City']
print(f'{best_change_city} is the city with the best overall pollution change over time with a pollutant density change of {best_change_c}.')


worst_change_c = city_total_df['Change_Over_Time'].max()
worst_change_city = city_total_df.loc[city_total_df['Change_Over_Time'].idxmax(), 'City']
print(f'{worst_change_city} is the city with the worst overall pollution change over time with a pollutant density change of {worst_change_c}.')

Billings is the city with the best overall pollution change over time with a pollutant density change of -309.0.
Eugene is the city with the worst overall pollution change over time with a pollutant density change of 364.68999999999994.


Now we will take a look at some statistics at the individual pollutant level.

In [81]:
# Which states have the lowest and highest O3 concentration?
low_O3 = O3_avg['2022'].min()
best_state_O3 = O3_avg.loc[O3_avg['2022'].idxmin(), 'State']
print(f'{best_state_O3} is the state with the lowest concentration of O3 with a value of {low_O3}')

high_O3 = O3_avg['2022'].max()
worst_state_O3 = O3_avg.loc[O3_avg['2022'].idxmax(), 'State']
print(f'{worst_state_O3} is the state with the highest concentration of O3 with a value of {high_O3}')

HI is the state with the lowest concentration of O3 with a value of 0.044
CT is the state with the highest concentration of O3 with a value of 0.07533333333333332
